In [34]:
DATABASE='googol'
HOST='127.0.0.1'
USERNAME='kenjiwang'
PASSWORD='5517007'
PORT='5432'

In [35]:
import torch
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer
from transformers.dynamic_module_utils import get_imports
from unittest.mock import patch
from numpy.linalg import norm
import numpy as np
import psycopg2

In [36]:
conn = psycopg2.connect(database=DATABASE,
	host=HOST,
	user=USERNAME,
	password=PASSWORD,
	port=PORT
)

cursor = conn.cursor()

In [40]:
model = SentenceTransformer('dunzhang/stella_en_1.5B_v5', trust_remote_code=True)

In [49]:
def search_googol(query):
	query_formatted = query.replace('\'', '\'\'')
	query_emb = model.encode([query], normalize_embeddings=True)[0].tolist()

	pg_query = f"SELECT url, title, description FROM websites ORDER BY description_emb <=> '{query_emb}', title_emb <=> '{query_emb}' LIMIT 10;"

	cursor.execute(pg_query)
	many_rows = cursor.fetchmany(10)

	for row in many_rows:
		print(row)

In [50]:
search_googol("British news")

('apple.com', 'Apple', 'Discover the innovative world of Apple and shop everything iPhone, iPad, Apple Watch, Mac, and Apple TV, plus explore accessories, entertainment, and expert device support.')
('apps.apple.com', 'App Store - Apple', 'The App Store gives people around the world a safe and trusted place to discover apps that meet our high standards for privacy, security, and content.')
('open.spotify.com', 'Spotify - Web Player: Music for everyone', 'Spotify is a digital music service that gives you access to millions of songs.')
('wordpress.org', 'Blog Tool, Publishing Platform, and CMS – WordPress.org', 'Open source software which you can use to easily create a beautiful website, blog, or app.')
('maps.google.com', '  Google Maps  ', 'Find local businesses, view maps and get driving directions in Google Maps.')
('pinterest.com', 'Pinterest', 'Discover recipes, home ideas, style inspiration and other ideas to try.')
('theguardian.com', 'Latest news, sport and opinion from the Guar